# The cartesian product problem

This notebook specifies what the cartesian product function is supposed to do by implementing it in a sequential (non-vectorized) way.

A bit of nomenclature: we had been calling it "combinations" because I misremembered the name from `itertools`. The algorithm I was thinking about is called `itertools.product` for "cartesian product"— `itertools.combinations` is something else. (Sorry!)

In [1]:
import itertools
help(itertools.product)

Help on class product in module itertools:

class product(__builtin__.object)
 |  product(*iterables) --> product object
 |  
 |  Cartesian product of input iterables.  Equivalent to nested for-loops.
 |  
 |  For example, product(A, B) returns the same as:  ((x,y) for x in A for y in B).
 |  The leftmost iterators are in the outermost for-loop, so the output tuples
 |  cycle in a manner similar to an odometer (with the rightmost element changing
 |  on every iteration).
 |  
 |  To compute the product of an iterable with itself, specify the number
 |  of repetitions with the optional repeat keyword argument. For example,
 |  product(A, repeat=4) means the same as product(A, A, A, A).
 |  
 |  product('ab', range(3)) --> ('a',0) ('a',1) ('a',2) ('b',0) ('b',1) ('b',2)
 |  product((0,1), (0,1), (0,1)) --> (0,0,0) (0,0,1) (0,1,0) (0,1,1) (1,0,0) ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __iter__(...)
 |      

**Example:**

In [2]:
list(itertools.product(["a", "b", "c"], ["x", "y"]))

[('a', 'x'), ('a', 'y'), ('b', 'x'), ('b', 'y'), ('c', 'x'), ('c', 'y')]

Python's `itertools.product` can iterate over collections of any type of object, but for vectorization, we'll have to limit our attention to numbers. In fact, if we limit ourselves to only products of `range`, then OAMap can pick that up and use it as pointer indexes, so for us, `itertools.product` of `range` *is* fully general.

In [3]:
indexes = list(itertools.product(range(3), range(2)))        # this is sufficient for us
indexes

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

In [4]:
collection1 = ["a", "b", "c"]
collection2 = ["x", "y"]
[(collection1[i1], collection2[i2]) for i1, i2 in indexes]   # because this is what OAMap pointer does

[('a', 'x'), ('a', 'y'), ('b', 'x'), ('b', 'y'), ('c', 'x'), ('c', 'y')]

# The physics case: products within events


The `itertools.product` function takes a cartesian product of a whole collection with another— a "full join" in SQL terms. Imagine collections with millions of items: the number of items in the cartesian product is the multiplicative product of the number of items in each collection, so the cartesian product would be on the order of trillions of items. It scales poorly.

Moreover, a full join is not what a physicist wants. Physicists need to compare, for example, all jets in an event with all muons in the same event, but never any combination of particles in different events. This dramatically cuts down the scale of the problem.

It also adds a new complication: we have to compute cartesian products *separately for each event.* This notebook will show the per-event product algorithm three different ways.

# Sample inputs

Here are some starts/stops arrays for two particle collections in a set of events. I'm using the "offsets" form, where

`starts, stops = offsets[:-1], offsets[1:]`

which represents slightly stronger assumptions than generic starts/stops (and is therefore easier). I believe all of this can be generalized to starts/stops, but `product` takes particle collections as a vararg (you can have arbitrarily many of them). Having a single array for each particle collection simplifies this argument handling— an issue that has nothing to do with the cartesian product algorithm, but the point of this is to be illustrative, so we want to avoid all unnecessary complications.

The two particle collections will have a Poisson-distributed number of particle per event— fairly realistic.

In [5]:
import numpy
import itertools

NUMEVENTS = 10

offsets1 = numpy.empty(NUMEVENTS + 1, dtype=int)                     # offsets has N+1 entries for N events
offsets1[0] = 0
offsets1[1:] = numpy.cumsum(numpy.random.poisson(1.5, NUMEVENTS))    # 1.5 particles per event on average

offsets2 = numpy.empty(NUMEVENTS + 1, dtype=int)
offsets2[0] = 0
offsets2[1:] = numpy.cumsum(numpy.random.poisson(2.5, NUMEVENTS))    # 2.5 particles per event on average

print(offsets1)
print(offsets2)

[ 0  2  2  4  4  6  9 11 14 15 15]
[ 0  2  5  9 10 13 14 17 17 20 22]


For each event, the number of particles of each type and their combinations is:

In [6]:
ntotcomb = 0
for i in range(NUMEVENTS):
    ntype1 = offsets1[i + 1] - offsets1[i]
    ntype2 = offsets2[i + 1] - offsets2[i]
    print("event {} has {} particles of type 1 and {} particles of type 2 => {} combinations".format(
        i, ntype1, ntype2, ntype1*ntype2))
    ntotcomb += ntype1*ntype2

print("=> {} entries in the output arrays".format(ntotcomb))

event 0 has 2 particles of type 1 and 2 particles of type 2 => 4 combinations
event 1 has 0 particles of type 1 and 3 particles of type 2 => 0 combinations
event 2 has 2 particles of type 1 and 4 particles of type 2 => 8 combinations
event 3 has 0 particles of type 1 and 1 particles of type 2 => 0 combinations
event 4 has 2 particles of type 1 and 3 particles of type 2 => 6 combinations
event 5 has 3 particles of type 1 and 1 particles of type 2 => 3 combinations
event 6 has 2 particles of type 1 and 3 particles of type 2 => 6 combinations
event 7 has 3 particles of type 1 and 0 particles of type 2 => 0 combinations
event 8 has 1 particles of type 1 and 3 particles of type 2 => 3 combinations
event 9 has 0 particles of type 1 and 2 particles of type 2 => 0 combinations
=> 30 entries in the output arrays


# Implementation 1: without using itertools.product

Using the function we want to illustrate in the implementation would be hiding the details, so let's implement the per-event product without using `itertools.product`. Remember that this function takes arbitrarily many particle collections as input (as `*offsets`).

In [7]:
def product(*offsets):
    assert len(offsets) >= 2

    # in a first pass, we have to determine how big the output is
    outcounts = numpy.ones(NUMEVENTS, dtype=int)          # outcounts is a per-event number of combinations
    for offset in offsets:
        counts = offset[1:] - offset[:-1]                 # counts is stops - starts
        outcounts *= counts
        print("counts     = {}".format(counts.tolist()))
    print("outcounts  = {}".format(outcounts.tolist()))

    # now turn that counts array into an offsets array via cumsum
    outoffsets = numpy.empty(NUMEVENTS + 1, dtype=int)
    outoffsets[0] = 0
    outoffsets[1:] = numpy.cumsum(outcounts)

    print("outoffsets = {}".format(outoffsets.tolist()))

    # one "out" array per particle type, but all of them have the same length: the number of combinations
    totalcount = outcounts.sum()
    outs = [numpy.empty(totalcount, dtype=int) for offset in offsets]

    # implementing "product" for an arbitrary number of inputs requires recursion (usually hard to vectorize)
    def rangeall(i, j, k):
        if j == len(offsets):
            return 1
        else:
            cumulative = 0
            for index in range(offsets[j][i], offsets[j][i + 1]):
                cumulant = rangeall(i, j + 1, cumulative)
                outs[j][outoffsets[i] + k + cumulative : outoffsets[i] + k + cumulative + cumulant] = index
                # print("outs[{j}][outoffsets[{i}] + {k} + {cumulative} : outoffsets[{i}] + {k} + {cumulative} + {cumulant}] = {index}".format(
                #     i=i, j=j, k=k, cumulative=cumulative, cumulant=cumulant, index=index))
                cumulative += cumulant
            return cumulative

    # the actual loop over events, which calls the recursive function once per event
    for i in range(NUMEVENTS):
        print("event {}:".format(i))
        rangeall(i, 0, 0)
        print("outs:\n    {}".format("\n    ".join(repr(out[outoffsets[i]:outoffsets[i + 1]].tolist()) for out in outs)))

    # we need to get out the offsets for the outs arrays and the outs arrays
    return outoffsets, outs

In [8]:
# compute it
outoffsets, outs = product(offsets1, offsets2)

counts     = [2, 0, 2, 0, 2, 3, 2, 3, 1, 0]
counts     = [2, 3, 4, 1, 3, 1, 3, 0, 3, 2]
outcounts  = [4, 0, 8, 0, 6, 3, 6, 0, 3, 0]
outoffsets = [0, 4, 4, 12, 12, 18, 21, 27, 27, 30, 30]
event 0:
outs:
    [0, 0, 1, 1]
    [0, 1, 0, 1]
event 1:
outs:
    []
    []
event 2:
outs:
    [2, 2, 2, 2, 3, 3, 3, 3]
    [5, 6, 7, 8, 5, 6, 7, 8]
event 3:
outs:
    []
    []
event 4:
outs:
    [4, 4, 4, 5, 5, 5]
    [10, 11, 12, 10, 11, 12]
event 5:
outs:
    [6, 7, 8]
    [13, 13, 13]
event 6:
outs:
    [9, 9, 9, 10, 10, 10]
    [14, 15, 16, 14, 15, 16]
event 7:
outs:
    []
    []
event 8:
outs:
    [14, 14, 14]
    [17, 18, 19]
event 9:
outs:
    []
    []


In [9]:
# and demonstrate that the output is what we want (equivalent to nested for loops)

def demonstrate(offsets1, offsets2, outoffsets, outs):
    leftcolumn = []
    for i in range(NUMEVENTS):
        leftcolumn.append("event {}".format(i))
        for j1 in range(offsets1[i], offsets1[i + 1]):
            for j2 in range(offsets2[i], offsets2[i + 1]):
                leftcolumn.append("{} {}".format(j1, j2))

    rightcolumn = []
    for outindex, (j1, j2) in enumerate(zip(outs[0], outs[1])):
        for i, offset in enumerate(outoffsets):
            if outindex == offset:
                rightcolumn.append("event {}".format(i))
        rightcolumn.append("{} {}".format(j1, j2))

    for left, right in zip(leftcolumn, rightcolumn):
        print("{:10s} {:10s}".format(left, right))

    for i, (left, right) in enumerate(zip(leftcolumn, rightcolumn)):
        assert left == right, "fails on line {}: {} != {}".format(i, left, right)

# print out what you get from for loops on the left, what you get from the outoffsets and outs on the right
demonstrate(offsets1, offsets2, outoffsets, outs)

event 0    event 0   
0 0        0 0       
0 1        0 1       
1 0        1 0       
1 1        1 1       
event 1    event 1   
event 2    event 2   
2 5        2 5       
2 6        2 6       
2 7        2 7       
2 8        2 8       
3 5        3 5       
3 6        3 6       
3 7        3 7       
3 8        3 8       
event 3    event 3   
event 4    event 4   
4 10       4 10      
4 11       4 11      
4 12       4 12      
5 10       5 10      
5 11       5 11      
5 12       5 12      
event 5    event 5   
6 13       6 13      
7 13       7 13      
8 13       8 13      
event 6    event 6   
9 14       9 14      
9 15       9 15      
9 16       9 16      
10 14      10 14     
10 15      10 15     
10 16      10 16     
event 7    event 7   
event 8    event 8   
14 17      14 17     
14 18      14 18     
14 19      14 19     


# Implementation 2: with itertools.product

That was a little hard to read because of the recursion. So let's put `itertools.product` back in. It's still useful to look at the algorithm this way because it makes a distinction between what is the event-handling and what is the cartesian product.

In [10]:
def product_itertools(*offsets):
    assert len(offsets) >= 2

    outcounts = numpy.ones(NUMEVENTS, dtype=int)
    for offset in offsets:
        counts = offset[1:] - offset[:-1]   # this is stops - starts
        outcounts *= counts
        print("counts     = {}".format(counts.tolist()))
    print("outcounts  = {}".format(outcounts.tolist()))

    outoffsets = numpy.empty(NUMEVENTS + 1, dtype=int)
    outoffsets[0] = 0
    outoffsets[1:] = numpy.cumsum(outcounts)

    print("outoffsets = {}".format(outoffsets.tolist()))

    totalcount = outcounts.sum()
    outs = [numpy.empty(totalcount, dtype=int) for offset in offsets]

    where = 0
    for i in range(NUMEVENTS):
        print("event {}:".format(i))
        for indexes in itertools.product(*[range(offset[i], offset[i + 1]) for offset in offsets]):
            for index, out in zip(indexes, outs):
                out[where] = index
            where += 1
        print("outs:\n    {}".format("\n    ".join(repr(out[outoffsets[i]:outoffsets[i + 1]].tolist()) for out in outs)))

    return outoffsets, outs

In [11]:
# compute it
outoffsets, outs = product_itertools(offsets1, offsets2)

counts     = [2, 0, 2, 0, 2, 3, 2, 3, 1, 0]
counts     = [2, 3, 4, 1, 3, 1, 3, 0, 3, 2]
outcounts  = [4, 0, 8, 0, 6, 3, 6, 0, 3, 0]
outoffsets = [0, 4, 4, 12, 12, 18, 21, 27, 27, 30, 30]
event 0:
outs:
    [0, 0, 1, 1]
    [0, 1, 0, 1]
event 1:
outs:
    []
    []
event 2:
outs:
    [2, 2, 2, 2, 3, 3, 3, 3]
    [5, 6, 7, 8, 5, 6, 7, 8]
event 3:
outs:
    []
    []
event 4:
outs:
    [4, 4, 4, 5, 5, 5]
    [10, 11, 12, 10, 11, 12]
event 5:
outs:
    [6, 7, 8]
    [13, 13, 13]
event 6:
outs:
    [9, 9, 9, 10, 10, 10]
    [14, 15, 16, 14, 15, 16]
event 7:
outs:
    []
    []
event 8:
outs:
    [14, 14, 14]
    [17, 18, 19]
event 9:
outs:
    []
    []


In [12]:
# and demonstrate that the output is what we want
demonstrate(offsets1, offsets2, outoffsets, outs)

event 0    event 0   
0 0        0 0       
0 1        0 1       
1 0        1 0       
1 1        1 1       
event 1    event 1   
event 2    event 2   
2 5        2 5       
2 6        2 6       
2 7        2 7       
2 8        2 8       
3 5        3 5       
3 6        3 6       
3 7        3 7       
3 8        3 8       
event 3    event 3   
event 4    event 4   
4 10       4 10      
4 11       4 11      
4 12       4 12      
5 10       5 10      
5 11       5 11      
5 12       5 12      
event 5    event 5   
6 13       6 13      
7 13       7 13      
8 13       8 13      
event 6    event 6   
9 14       9 14      
9 15       9 15      
9 16       9 16      
10 14      10 14     
10 15      10 15     
10 16      10 16     
event 7    event 7   
event 8    event 8   
14 17      14 17     
14 18      14 18     
14 19      14 19     


# Implementation 3: only two inputs

To further simplify the algorithm, suppose there are only two inputs. Then we can put explicit for loops in the generation of the output.

In [13]:
def product_two(offsets1, offsets2):
    counts1 = offsets1[1:] - offsets1[:-1]
    print("counts     = {}".format(counts1.tolist()))
    counts2 = offsets2[1:] - offsets2[:-1]
    print("counts     = {}".format(counts2.tolist()))

    outcounts = counts1 * counts2
    print("outcounts  = {}".format(outcounts.tolist()))

    outoffsets = numpy.empty(NUMEVENTS + 1, dtype=int)
    outoffsets[0] = 0
    outoffsets[1:] = numpy.cumsum(outcounts)

    print("outoffsets = {}".format(outoffsets.tolist()))

    totalcount = outcounts.sum()
    outs1 = numpy.empty(totalcount, dtype=int)
    outs2 = numpy.empty(totalcount, dtype=int)

    where = 0
    for i in range(NUMEVENTS):
        print("event {}".format(i))
        for index1 in range(offsets1[i], offsets1[i + 1]):
            for index2 in range(offsets2[i], offsets2[i + 1]):
                outs1[where] = index1
                outs2[where] = index2
                where += 1
        print("outs:\n    {}\n    {}".format(outs1[outoffsets[i]:outoffsets[i + 1]], outs2[outoffsets[i]:outoffsets[i + 1]]))

    return outoffsets, [outs1, outs2]

In [14]:
# compute it
outoffsets, outs = product_two(offsets1, offsets2)

counts     = [2, 0, 2, 0, 2, 3, 2, 3, 1, 0]
counts     = [2, 3, 4, 1, 3, 1, 3, 0, 3, 2]
outcounts  = [4, 0, 8, 0, 6, 3, 6, 0, 3, 0]
outoffsets = [0, 4, 4, 12, 12, 18, 21, 27, 27, 30, 30]
event 0
outs:
    [0 0 1 1]
    [0 1 0 1]
event 1
outs:
    []
    []
event 2
outs:
    [2 2 2 2 3 3 3 3]
    [5 6 7 8 5 6 7 8]
event 3
outs:
    []
    []
event 4
outs:
    [4 4 4 5 5 5]
    [10 11 12 10 11 12]
event 5
outs:
    [6 7 8]
    [13 13 13]
event 6
outs:
    [ 9  9  9 10 10 10]
    [14 15 16 14 15 16]
event 7
outs:
    []
    []
event 8
outs:
    [14 14 14]
    [17 18 19]
event 9
outs:
    []
    []


In [15]:
# and demonstrate that the output is what we want
demonstrate(offsets1, offsets2, outoffsets, outs)

event 0    event 0   
0 0        0 0       
0 1        0 1       
1 0        1 0       
1 1        1 1       
event 1    event 1   
event 2    event 2   
2 5        2 5       
2 6        2 6       
2 7        2 7       
2 8        2 8       
3 5        3 5       
3 6        3 6       
3 7        3 7       
3 8        3 8       
event 3    event 3   
event 4    event 4   
4 10       4 10      
4 11       4 11      
4 12       4 12      
5 10       5 10      
5 11       5 11      
5 12       5 12      
event 5    event 5   
6 13       6 13      
7 13       7 13      
8 13       8 13      
event 6    event 6   
9 14       9 14      
9 15       9 15      
9 16       9 16      
10 14      10 14     
10 15      10 15     
10 16      10 16     
event 7    event 7   
event 8    event 8   
14 17      14 17     
14 18      14 18     
14 19      14 19     


# Pairs from product

I said at a meeting that the `pairs` algorithm is easily derivable from the `product` algorithm (which I was calling "combinations" at that time). The `pairs` algorithm takes one collection as input, passes it to `product` twice, and masks out cases in which `index1 <= index2`.

In [16]:
def pairs(offsets):
    outoffsets, outs = product_two(offsets, offsets)
    mask = outs[1] > outs[0]    # excludes diagonal of matrix; a useful variant keeps the diagonal
    return outoffsets, outs[0], outs[1], mask

outoffsets, outs1, outs2, mask = pairs(offsets1)

counts     = [2, 0, 2, 0, 2, 3, 2, 3, 1, 0]
counts     = [2, 0, 2, 0, 2, 3, 2, 3, 1, 0]
outcounts  = [4, 0, 4, 0, 4, 9, 4, 9, 1, 0]
outoffsets = [0, 4, 4, 8, 8, 12, 21, 25, 34, 35, 35]
event 0
outs:
    [0 0 1 1]
    [0 1 0 1]
event 1
outs:
    []
    []
event 2
outs:
    [2 2 3 3]
    [2 3 2 3]
event 3
outs:
    []
    []
event 4
outs:
    [4 4 5 5]
    [4 5 4 5]
event 5
outs:
    [6 6 6 7 7 7 8 8 8]
    [6 7 8 6 7 8 6 7 8]
event 6
outs:
    [ 9  9 10 10]
    [ 9 10  9 10]
event 7
outs:
    [11 11 11 12 12 12 13 13 13]
    [11 12 13 11 12 13 11 12 13]
event 8
outs:
    [14]
    [14]
event 9
outs:
    []
    []


In [17]:
# and demonstrate that it's what we want

def demonstrate_pairs(offsets, outoffsets, outs1, outs2, mask):
    leftcolumn = []
    for i in range(NUMEVENTS):
        leftcolumn.append("event {}".format(i))
        for j1 in range(offsets[i], offsets[i + 1]):
            for j2 in range(j1 + 1, offsets[i + 1]):
                leftcolumn.append("{} {}".format(j1, j2))

    rightcolumn = []
    for outindex, (j1, j2, m) in enumerate(zip(outs1, outs2, mask)):
        for i, offset in enumerate(outoffsets):
            if outindex == offset:
                rightcolumn.append("event {}".format(i))
        if m:
            rightcolumn.append("{} {}".format(j1, j2))

    for left, right in zip(leftcolumn, rightcolumn):
        print("{:10s} {:10s}".format(left, right))

    for i, (left, right) in enumerate(zip(leftcolumn, rightcolumn)):
        assert left == right, "fails on line {}: {} != {}".format(i, left, right)

demonstrate_pairs(offsets1, outoffsets, outs1, outs2, mask)

event 0    event 0   
0 1        0 1       
event 1    event 1   
event 2    event 2   
2 3        2 3       
event 3    event 3   
event 4    event 4   
4 5        4 5       
event 5    event 5   
6 7        6 7       
6 8        6 8       
7 8        7 8       
event 6    event 6   
9 10       9 10      
event 7    event 7   
11 12      11 12     
11 13      11 13     
12 13      12 13     
event 8    event 8   
